In [27]:
"""
Magic commands
"""
%matplotlib inline
%load_ext autoreload
%autoreload 2

"""
Python standard packages
"""
import os
import sys

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

"""
Own packages
"""
sys.path.insert(0, "C:\\code\\speciale\\") # vesl path
sys.path.insert(0, "C:\\Users\\lucas\\OneDrive\\Skrivebord\\repo\\speciale\\") # ljb path

from market_simulation_study.agent import RandomAgent, InvestorAgent, TrendAgent, RLAgent, MarketMakerAgent
from market_simulation_study.environment import MarketEnvironment

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [197]:
# Declare parameters
n_random_agents= 2
n_investor_agents = 0
n_trend_agents = 2
n_rl_agents = 0

n_agents = n_random_agents + n_investor_agents + n_trend_agents + n_rl_agents


time_periods = 50

price_list = [100 + np.random.normal() for j in range(100)]
ex_list = np.array([[0]*n_agents, [0]*n_agents])
fee = 0
slippage = 0
mean_buy_price = 99
mean_sell_price = 101

state0 = {"market_prices": price_list, "execution_status": ex_list, 'fee':fee, "mean_buy_price":mean_buy_price,
          "mean_sell_price":mean_sell_price, 'slippage':slippage}

env = MarketEnvironment(state0)
agents = []

random_agents = [RandomAgent(agent_id = j,
                             delta = np.random.normal()) for j in range(n_random_agents)]

investor_agents = [InvestorAgent(agent_id = j + n_random_agents,
                                 delta = np.random.normal(),
                                 intensity = 0.1) for j in range(n_investor_agents)]

trend_agents = [TrendAgent(agent_id = j + n_random_agents + n_investor_agents,
                                 delta = np.random.normal()) for j in range(n_trend_agents)]

rl_agents = [RLAgent(agent_id = j + n_random_agents + n_investor_agents + n_trend_agents,
                                 delta = np.random.normal()) for j in range(n_rl_agents)]

agents.append(random_agents)
agents.append(investor_agents)
agents.append(trend_agents)
agents.append(rl_agents)

agents = [item for sublist in agents for item in sublist]

# Initialize agents
for j in range(n_agents):
        agents[j].update(state0)

In [208]:
# INITIALIZE ORDER BOOKS
sell_order_book = agents[0].sell_order
buy_order_book = agents[0].buy_order
sell_order_book = pd.DataFrame(sell_order_book, index = sell_order_book.iloc[:, -1])
buy_order_book = pd.DataFrame(buy_order_book, index = buy_order_book.iloc[:, -1])

for i in range(len(agents)-1):
    #========================================#
    # CHECK IF AGENT i CAN MAKE A BUY TRADE #
    #========================================#
    if any(agents[i+1].buy_order["buy_price"].values >= sell_order_book.iloc[:, 0].values):
        print("price_match: ", agents[i+1].buy_order["buy_price"].values, sell_order_book.iloc[:, 0].values)
        
        matched_order_book = sell_order_book[sell_order_book["sell_price"].values <= agents[i+1].buy_order["buy_price"].values]
        matched_order_book = matched_order_book.sort_values(["sell_price", "latency"], ascending = [True, True])
        display(sell_order_book)
        display(agents[i+1].buy_order)
        for index, order in matched_order_book.iterrows():
            if agents[i+1].buy_order["buy_volume"].values > order["sell_volume"]:
                trade_volume = order["sell_volume"].copy()
            else:
                trade_volume = agents[i+1].buy_order["buy_volume"].values.copy()
            
            agents[i+1].buy_order["buy_volume"] -= trade_volume
            agents[int(order["agent_id"])].sell_order["sell_volume"] -= trade_volume
            sell_order_book.at[index, 'sell_volume'] -= trade_volume
            
            agents[i+1].position += trade_volume
            agents[int(order["agent_id"])].position -= trade_volume
            
    # SELL ORDER INTO SELL ORDER
    sell_order_book = sell_order_book[sell_order_book["sell_volume"] > 0]
    if agents[i+1].buy_order["buy_volume"].values > 0:
        buy_order_book = buy_order_book.append(agents[i+1].buy_order)
    
    sell_order_book = pd.DataFrame(sell_order_book, index = sell_order_book.iloc[:, -1])
    buy_order_book = pd.DataFrame(buy_order_book, index = buy_order_book.iloc[:, -1])
    
    #========================================#
    # CHECK IF AGENT i CAN MAKE A SELL TRADE #
    #========================================#
    if any(agents[i+1].sell_order["sell_price"].values <= buy_order_book.iloc[:, 0].values):
        print("price_match: ", agents[i+1].sell_order["sell_price"].values, buy_order_book.iloc[:, 0].values)
        
        matched_order_book = buy_order_book[buy_order_book["buy_price"].values >= agents[i+1].sell_order["sell_price"].values]
        matched_order_book = matched_order_book.sort_values(["buy_price", "latency"], ascending = [True, True])
        display(matched_order_book)
        for index, order in matched_order_book.iterrows():
            if agents[i+1].sell_order["sell_volume"].values > order["buy_volume"]:
                trade_volume = order["buy_volume"].copy()
            else:
                trade_volume = agents[i+1].sell_order["sell_volume"].values.copy()
            print(sell_order_book, " INDEX: ", index, " AGENT ID: ", order["agent_id"])
            
            agents[i+1].sell_order["sell_volume"] -= trade_volume
            agents[int(order["agent_id"])].buy_order["buy_volume"] -= trade_volume
            buy_order_book.at[index, 'buy_volume'] -= trade_volume
            
            agents[i+1].position -= trade_volume
            agents[int(order["agent_id"])].position = trade_volume
            # agents[sob["ID"]].all_trades.append([order["BP"], - trade_volume])

    # buy ORDER INTO buy ORDER
    buy_order_book = buy_order_book[buy_order_book["buy_volume"] > 0]
    if agents[i+1].sell_order["sell_volume"].values > 0:
        sell_order_book = sell_order_book.append(agents[i+1].sell_order)    
    
    sell_order_book = pd.DataFrame(sell_order_book, index = sell_order_book.iloc[:, -1])
    buy_order_book = pd.DataFrame(buy_order_book, index = buy_order_book.iloc[:, -1])


In [204]:
agents[1].position

-1.0

In [209]:
display(buy_order_book)
display(sell_order_book)

,buy_price,buy_volume,latency,agent_id
agent_id,,,,
0.0,98.466502,1.0,0.984619,0.0
1.0,96.864411,1.0,0.580663,1.0
2.0,105.220009,5.0,2.441061,2.0
3.0,107.921702,3.0,-0.965091,3.0


,sell_price,sell_volume,latency,agent_id
agent_id,,,,


In [210]:
display(agents[0].sell_order)
display(agents[1].sell_order)
display(agents[2].sell_order)
display(agents[3].sell_order)
display(agents[0].buy_order)
display(agents[1].buy_order)
display(agents[2].buy_order)
display(agents[3].buy_order)

,sell_price,sell_volume,latency,agent_id
0,110.508789,0.0,0.984619,0.0


,sell_price,sell_volume,latency,agent_id
1,107.524955,0.0,0.580663,1.0


,sell_price,sell_volume,latency,agent_id
2,NaN,0.0,2.441061,2.0


,sell_price,sell_volume,latency,agent_id
3,NaN,0.0,-0.965091,3.0


,buy_price,buy_volume,latency,agent_id
0,98.466502,1.0,0.984619,0.0


,buy_price,buy_volume,latency,agent_id
1,96.864411,1.0,0.580663,1.0


,buy_price,buy_volume,latency,agent_id
2,105.220009,5.0,2.441061,2.0


,buy_price,buy_volume,latency,agent_id
3,107.921702,3.0,-0.965091,3.0
